In [1]:
import io
import pandas as pd
import requests

In [113]:
# Inserting File via URL

url = 'https://storage.googleapis.com/uber_data_engineer_project/uber_data.csv'
response = requests.get(url)

In [114]:
df = pd.read_csv(io.StringIO(response.text), sep=',')

In [115]:
# CREATING DATETIME_DIM TABLE AS PER DATA MODEL
# Changing Object type to datetime

df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

In [116]:
# Removing duplicates and marking "trip_id" as index

df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [117]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
0,1,2016-03-01,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,0
1,1,2016-03-01,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,1
2,2,2016-03-01,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,2
3,2,2016-03-01,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,3
4,2,2016-03-01,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,4


In [121]:
# Extracting pick_ip hour, day, month and year from datetime column

datetime_dim = df[["tpep_pickup_datetime", "tpep_dropoff_datetime"]].reset_index(drop=True)
datetime_dim["tpep_pickup_datetime"] = datetime_dim["tpep_pickup_datetime"]
datetime_dim["pick_hour"] = datetime_dim["tpep_pickup_datetime"].dt.hour
datetime_dim["pick_day"] = datetime_dim["tpep_pickup_datetime"].dt.day
datetime_dim["pick_month"] = datetime_dim["tpep_pickup_datetime"].dt.month
datetime_dim["pick_year"] = datetime_dim["tpep_pickup_datetime"].dt.year
datetime_dim["pick_weekday"] = datetime_dim["tpep_pickup_datetime"].dt.weekday

In [122]:
# Extracting drop hour, day, month and year from datetime column

datetime_dim["tpep_dropoff_datetime"] = datetime_dim["tpep_dropoff_datetime"]
datetime_dim["drop_hour"] = datetime_dim["tpep_dropoff_datetime"].dt.hour
datetime_dim["drop_day"] = datetime_dim["tpep_dropoff_datetime"].dt.day
datetime_dim["drop_month"] = datetime_dim["tpep_dropoff_datetime"].dt.month
datetime_dim["drop_year"] = datetime_dim["tpep_dropoff_datetime"].dt.year
datetime_dim["drop_weekday"] = datetime_dim["tpep_dropoff_datetime"].dt.weekday

In [123]:
datetime_dim

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,2016-03-01 00:00:00,2016-03-01 00:07:55,0,1,3,2016,1,0,1,3,2016,1
1,2016-03-01 00:00:00,2016-03-01 00:11:06,0,1,3,2016,1,0,1,3,2016,1
2,2016-03-01 00:00:00,2016-03-01 00:31:06,0,1,3,2016,1,0,1,3,2016,1
3,2016-03-01 00:00:00,2016-03-01 00:00:00,0,1,3,2016,1,0,1,3,2016,1
4,2016-03-01 00:00:00,2016-03-01 00:00:00,0,1,3,2016,1,0,1,3,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2016-03-01 06:17:10,2016-03-01 06:22:15,6,1,3,2016,1,6,1,3,2016,1
99996,2016-03-01 06:17:10,2016-03-01 06:32:41,6,1,3,2016,1,6,1,3,2016,1
99997,2016-03-01 06:17:10,2016-03-01 06:37:23,6,1,3,2016,1,6,1,3,2016,1
99998,2016-03-01 06:17:10,2016-03-01 06:22:09,6,1,3,2016,1,6,1,3,2016,1


In [124]:
datetime_dim.index

RangeIndex(start=0, stop=100000, step=1)

In [125]:
# Adding datetime_id as a column to use as a primary key in DB

datetime_dim["datetime_id"] = datetime_dim.index

In [126]:
# Arranging columns in table

datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [127]:
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-03-01,0,1,3,2016,1,2016-03-01 00:07:55,0,1,3,2016,1
1,1,2016-03-01,0,1,3,2016,1,2016-03-01 00:11:06,0,1,3,2016,1
2,2,2016-03-01,0,1,3,2016,1,2016-03-01 00:31:06,0,1,3,2016,1
3,3,2016-03-01,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1
4,4,2016-03-01,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1


In [128]:
# CREATING PASSENGER COUNT TABLE AS PER DATA MODEL

passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

In [129]:
passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,1
1,1,1
2,2,2
3,3,3
4,4,5


In [130]:
# CREATING TRIP_DISTANCE_DIM TABLE AS PER DATA MODEL

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [131]:
trip_distance_dim.head()

,trip_distance_id,trip_distance
0,0,2.50
1,1,2.90
2,2,19.98
3,3,10.78
4,4,30.43


In [132]:
# CREATING PICKUP_LOCATION_DIM TABLE AS PER DATA MODEL

pickup_location_dim = df[["pickup_longitude", "pickup_latitude"]].reset_index(drop=True)
pickup_location_dim["pickup_location_id"] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[["pickup_location_id", "pickup_longitude", "pickup_latitude"]]

In [133]:
pickup_location_dim

,pickup_location_id,pickup_longitude,pickup_latitude
0,0,-73.976746,40.765152
1,1,-73.983482,40.767925
2,2,-73.782021,40.644810
3,3,-73.863419,40.769814
4,4,-73.971741,40.792183
...,...,...,...
99995,99995,-73.990898,40.750519
99996,99996,-74.014488,40.718296
99997,99997,-73.963379,40.774097
99998,99998,-73.984901,40.763111


In [134]:
# CREATING DROPOFF_LOCATION_DIM TABLE AS PER DATA MODEL

dropoff_location_dim = df[["dropoff_longitude", "dropoff_latitude"]].reset_index(drop=True)
dropoff_location_dim["dropoff_location_id"] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[["dropoff_location_id", "dropoff_longitude", "dropoff_latitude"]]

In [135]:
dropoff_location_dim

,dropoff_location_id,dropoff_longitude,dropoff_latitude
0,0,-74.004265,40.746128
1,1,-74.005943,40.733166
2,2,-73.974541,40.675770
3,3,-73.969650,40.757767
4,4,-74.177170,40.695053
...,...,...,...
99995,99995,-73.998245,40.750462
99996,99996,-73.982361,40.752529
99997,99997,-73.865028,40.770512
99998,99998,-73.970695,40.759148


In [136]:
# CREATING RATE_CODE_DIM TABLE AS PER DATA MODEL

rate_code_type = {
    1: "Standard rate",
    2: "JFK",
    3: "Newark",
    4: "Nassau or Westchester",
    5: "Negotiated fare",
    6: "Group ride"
}

rate_code_dim = df[["RatecodeID"]].reset_index(drop=True)
rate_code_dim["rate_code_id"] = rate_code_dim.index
rate_code_dim["rate_code_name"] = rate_code_dim["RatecodeID"].map(rate_code_type)
rate_code_dim = rate_code_dim[["rate_code_id", "RatecodeID", "rate_code_name"]]

In [137]:
rate_code_dim

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark
...,...,...,...
99995,99995,1,Standard rate
99996,99996,1,Standard rate
99997,99997,1,Standard rate
99998,99998,1,Standard rate


In [138]:
# CREATING PAYMENT_TYPE_DIM TABLE AS PER DATA MODEL

payment_code_type = {
    1: "Credit Card",
    2: "Cash",
    3: "No charge",
    4: "Dispute",
    5: "Unknown",
    6: "Voided trip"
}

payment_type_dim = df[["payment_type"]].reset_index(drop=True)
payment_type_dim["payment_type_id"] = payment_type_dim.index
payment_type_dim["payment_type_name"] = payment_type_dim["payment_type"].map(payment_code_type)
payment_type_dim = payment_type_dim[["payment_type_id","payment_type", "payment_type_name"]]

In [139]:
payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit Card
1,1,1,Credit Card
2,2,1,Credit Card
3,3,1,Credit Card
4,4,1,Credit Card
...,...,...,...
99995,99995,2,Cash
99996,99996,1,Credit Card
99997,99997,1,Credit Card
99998,99998,1,Credit Card


In [140]:
# CREATING FACT TABLE AS PER DATA MODEL

fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [141]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,99995,99995,99995,99995,N,99995,99995,99995,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,99996,1,99996,99996,99996,99996,N,99996,99996,99996,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,99997,1,99997,99997,99997,99997,N,99997,99997,99997,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,99998,2,99998,99998,99998,99998,N,99998,99998,99998,5.5,0.5,0.5,1.36,0.00,0.3,8.16


In [142]:
fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trip_id                100000 non-null  int64  
 1   VendorID               100000 non-null  int64  
 2   datetime_id            100000 non-null  int64  
 3   passenger_count_id     100000 non-null  int64  
 4   trip_distance_id       100000 non-null  int64  
 5   rate_code_id           100000 non-null  int64  
 6   store_and_fwd_flag     100000 non-null  object 
 7   pickup_location_id     100000 non-null  int64  
 8   dropoff_location_id    100000 non-null  int64  
 9   payment_type_id        100000 non-null  int64  
 10  fare_amount            100000 non-null  float64
 11  extra                  100000 non-null  float64
 12  mta_tax                100000 non-null  float64
 13  tip_amount             100000 non-null  float64
 14  tolls_amount           100000 non-nul